In [2]:
# Set up libraries
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
from configparser import ConfigParser
from datetime import datetime, timedelta

In [3]:
# Configure the connection
config = ConfigParser()
config.read("config.ini")
options = config["DataBase"]

engine = create_engine("mysql://" + options["user"] + ":" + options["passwd"] + "@" + options["host"] + "/" + options["database"])
engine.connect()

In [4]:
station = "SELECT number FROM dublinbikes.static_data"
stat_df = pd.read_sql(station,engine)

In [10]:
for station in stat_df.number:
    # Set up the SQL command and turn into dataframe
    bikes_data = """SELECT bd.number, bd.last_update, bd.available_bikes, wa.weather_station
                FROM dublinbikes.bike_weather_assoc wa, dublinbikes.dynamic_data bd
                WHERE bd.number=wa.bike_station_id and bd.number = {} and bd.last_update >= 1584316800000;""".format(station)
    bikes_df = pd.read_sql(bikes_data, engine)
    weather_data = "SELECT name, weather_id, main_temp, main_humidity, wind_speed, dt FROM dublinbikes.weather_dynamic WHERE name = '{}';".format(bikes_df.weather_station[0])
    weather_df = pd.read_sql(weather_data, engine)
    ## cleaning bikes data
    # converting 'last_update' enties from 13 digits to the required 10
    for i in range(bikes_df.shape[0]):
        convert = str(bikes_df['last_update'][i])
        if len(convert) > 10:
            bikes_df.at[i,'last_update'] = int(convert[:10])
    # removing duplicates
    bikes_df = bikes_df.drop_duplicates()
    bikes_df = bikes_df.reset_index(drop=True)
    # convert update to datetime
    bikes_df['last_update'] = bikes_df['last_update'].apply(lambda x: datetime.fromtimestamp(x))
    # adding day of the week column
    bikes_df.insert(2, 'day_of_week', 0)
    bikes_df['day_of_week'] = bikes_df['last_update'].apply(lambda x: x.dayofweek )
    bikes_df.insert(2, 'hour_of_day', 0)
    bikes_df['hour_of_day'] = bikes_df['last_update'].apply(lambda x: x.hour )
    ## cleaning weather data
    # removing duplicates
    weather_df = weather_df.drop_duplicates()
    weather_df = weather_df.reset_index(drop=True)
    # convert update to datetime
    weather_df['dt'] = weather_df['dt'].apply(lambda x: datetime.fromtimestamp(x))
    ## creating the new combined table with default values
    combined_df = bikes_df
    # space for weather data
    combined_df = combined_df.assign(weather_id=0, main_temp=0, main_humidity=0,wind_speed=0, weather_time=datetime.fromtimestamp(1))
    # space for one hot encoding
    combined_df = combined_df.assign(d1=0,d2=0,d3=0,h1=0,h2=0,h3=0,h4=0,h5=0)
    ## joing both tables based on closest weather time(+- 6 mins)
    # start by removing rows that are dated before the earliest date time of the weather data
    combined_df = combined_df[combined_df.last_update>= (weather_df.dt[0] - timedelta(minutes = 10) )]
    combined_df = combined_df.reset_index(drop=True)
    # combine the closest weather date row with the corrisponding bike row 
    w_count = 0
    for i in range(combined_df.shape[0]):
        # current date to check against
        c_date = combined_df.last_update[i]
        # used in tracking the last timedelta checked
        try:
            time_deltaL = abs(c_date - weather_df.dt[w_count+1])
        except:
            time_deltaL = abs(c_date - weather_df.dt[w_count])
        # used in tracking the best time delta in the loop
        best = 0
        # check the next rows for as long as the time
        for c in range(1, weather_df.shape[0]-w_count):
            # the current best time delta bike_date-weather_date (closest weather date available)
            time_deltaC = abs(c_date - weather_df.dt[w_count+best])
            # the next time delta controlled by loop
            time_deltaN = abs(c_date - weather_df.dt[w_count+c])
            # if current best td < next td (next row)
            if time_deltaC < time_deltaN:
                # do nothing
                pass
            else:
                # make best this row
                best = c
            # if the current next td is bigger than the last loops td
            if time_deltaN > time_deltaL:
                # exit the loop and prevent going through every itereation
                # e.g if the best was updated on the last loop but this loop has
                # a bigger td then exit as the ones that follow will keep getting bigger
                break
            else:
                # update last td
                time_deltaL = time_deltaN
        # update the row count on the weather
        w_count += best
        # populate current row with wether data
        combined_df.at[i,'weather_id'] = weather_df.weather_id[w_count]
        combined_df.at[i,'main_temp'] = weather_df.main_temp[w_count]
        combined_df.at[i,'main_humidity'] = weather_df.main_humidity[w_count]
        combined_df.at[i,'wind_speed'] = weather_df.wind_speed[w_count]
        combined_df.at[i,'weather_time'] = weather_df.dt[w_count]
        # populate the current row with OHE data from day and hour
        day_bin = '{0:03b}'.format(combined_df.day_of_week[i])
        hour_bin = '{0:05b}'.format(combined_df.hour_of_day[i])
        combined_df.at[i,'d1'] = int(day_bin[-1])
        combined_df.at[i,'d2'] = int(day_bin[-2])
        combined_df.at[i,'d3'] = int(day_bin[-3])
        combined_df.at[i,'h1'] = int(hour_bin[-1])
        combined_df.at[i,'h2'] = int(hour_bin[-2])
        combined_df.at[i,'h3'] = int(hour_bin[-3])
        combined_df.at[i,'h4'] = int(hour_bin[-4])
        combined_df.at[i,'h5'] = int(hour_bin[-5])
    ## preparing data
    cont_features = ['weather_id', 'main_temp', 'main_humidity','wind_speed','d1','d2','d3','h1','h2','h3','h4','h5']
    features = combined_df[cont_features]
    target = combined_df.available_bikes
    # use all of the data to build the model for all future predictions
    hour_day_linreg = LinearRegression().fit(features, target)
    with open('models/dBikePredictCovData_station_{}.pkl'.format(station), 'wb') as handle:
        pickle.dump(hour_day_linreg, handle, pickle.HIGHEST_PROTOCOL)
